In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
X_train = np.load("/content/drive/MyDrive/ICPS/new_train_X_esm.npy")
y_train = np.load("/content/drive/MyDrive/ICPS/new_train_y_esm.npy")
X_test = np.load("/content/drive/MyDrive/ICPS/new_test_X_esm.npy")
y_test = np.load("/content/drive/MyDrive/ICPS/new_test_y_esm.npy")

Define Hybrid model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dropout, BatchNormalization
from tensorflow.keras.layers import Bidirectional, LSTM, GRU, Flatten, GlobalAveragePooling1D
from tensorflow.keras.layers import Dense, concatenate
from tensorflow.keras.optimizers import Adam

def Classifier_CNN_BiLSTM_Optimized(MAX_SEQUENCE_LENGTH, feature_dim, lr=1e-3):
    input1 = Input(shape=(MAX_SEQUENCE_LENGTH, feature_dim))

    # CNN Branch
    cnn = Conv1D(64, 3, activation="relu", padding="same")(input1)
    cnn = BatchNormalization()(cnn)
    cnn = MaxPooling1D()(cnn)
    cnn = Dropout(0.4)(cnn)
    cnn = Conv1D(128, 5, activation="relu", padding="same")(cnn)
    cnn = BatchNormalization()(cnn)
    cnn = MaxPooling1D()(cnn)
    cnn = Dropout(0.4)(cnn)
    cnn = Bidirectional(GRU(32, dropout=0.4, return_sequences=True))(cnn)
    cnn = GlobalAveragePooling1D()(cnn)
    cnn_branch = Flatten()(cnn)

    # BiLSTM Branch
    input2 = Input(shape=(MAX_SEQUENCE_LENGTH, feature_dim))
    lstm = Dropout(0.4)(input2)
    lstm = Bidirectional(LSTM(64, return_sequences=True))(lstm)
    lstm_branch = Flatten()(lstm)

    # Merge
    merged = concatenate([cnn_branch, lstm_branch])
    dense = Dense(256, activation="relu")(merged)
    dense = Dropout(0.4)(dense)
    output = Dense(1, activation="sigmoid")(dense)

    model = Model(inputs=[input1, input2], outputs=output)
    model.compile(optimizer=Adam(learning_rate=lr),
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.AUC()])
    return model

In [ ]:
model = Classifier_CNN_BiLSTM_Optimized(MAX_SEQUENCE_LENGTH, feature_dim)
# Hiển thị kiến trúc model
model.summary()

In [ ]:
model = Classifier_CNN_BiLSTM_Optimized(MAX_SEQUENCE_LENGTH)
# Huấn luyện
history = model.fit(
    [X_train, X_train],
    y_train,
    validation_data=([X_test, X_test], y_test),
    epochs=50,
    batch_size=batch
)

In [ ]:
model.save("/content/drive/MyDrive/ICPS/ICP_CNN_BiLSTM.h5")
